## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-21-48-47 +0000,nyt,Trump Says Gaza Talks Are Going ‘Very Well’ an...,https://www.nytimes.com/2025/10/08/us/politics...
1,2025-10-08-21-48-00 +0000,wsj,Opinion | The Incredible Expanding Fannie Mae,https://www.wsj.com/opinion/fannie-mae-freddie...
2,2025-10-08-21-45-13 +0000,nyt,Trump Calls Off Diplomatic Outreach to Venezuela,https://www.nytimes.com/2025/10/06/us/politics...
3,2025-10-08-21-44-34 +0000,bbc,Progress in Gaza peace talks as Trump says 've...,https://www.bbc.com/news/articles/cdjzvlxp8mjo...
4,2025-10-08-21-44-00 +0000,wsj,Opinion | More Marijuana Users Are Crash Dummies,https://www.wsj.com/opinion/marijuana-car-cras...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,48
67,new,23
1,gaza,15
123,shutdown,14
33,fire,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
99,2025-10-08-18-33-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...,105
114,2025-10-08-17-33-40 +0000,nyt,Trump Says Chicago Mayor and Illinois Governor...,https://www.nytimes.com/2025/10/08/us/trump-pr...,101
326,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...,99
72,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Illin...,https://www.latimes.com/politics/story/2025-10...,98
18,2025-10-08-21-31-13 +0000,cbc,Trump hints Mideast deal is 'very close' as se...,https://www.cbc.ca/news/world/trump-mideast-ga...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
99,105,2025-10-08-18-33-00 +0000,wsj,President Trump called for the jailing of Chic...,https://www.wsj.com/politics/policy/trump-call...
188,68,2025-10-08-13-33-06 +0000,nypost,Ex-FBI Director James Comey insists he’s innoc...,https://nypost.com/2025/10/08/us-news/ex-fbi-d...
74,65,2025-10-08-19-34-04 +0000,nypost,House Speaker Mike Johnson rejects standalone ...,https://nypost.com/2025/10/08/us-news/house-sp...
14,61,2025-10-08-21-33-46 +0000,nypost,Andrew Cuomo would scrap plan to close Rikers ...,https://nypost.com/2025/10/08/us-news/andrew-c...
66,56,2025-10-08-20-00-18 +0000,latimes,'Why wasn't the fire put out on the 1st?' Caus...,https://www.latimes.com/california/story/2025-...
326,51,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...
87,40,2025-10-08-19-09-53 +0000,bbc,Spy case collapse blamed on UK government fail...,https://www.bbc.com/news/articles/cy8rl7e7xp3o...
138,37,2025-10-08-16-28-37 +0000,nypost,Dad accused of killing girlfriend had sex with...,https://nypost.com/2025/10/08/world-news/dad-a...
317,36,2025-10-08-00-44-44 +0000,nyt,Athens Democracy Forum: As Democracy Erodes Gl...,https://www.nytimes.com/2025/10/07/world/europ...
72,35,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Illin...,https://www.latimes.com/politics/story/2025-10...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
